In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nestle_tasks').getOrCreate()

24/02/01 11:37:38 WARN Utils: Your hostname, venkatesh-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/02/01 11:37:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/01 11:37:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv('nestle_data.csv', header=True)

In [3]:
df.count()

10000

In [4]:
df_large = df.filter((df['SubChannelDesc']=='01.Large Provision')&(df['Final_Rank']<=8)).filter((df['SKUDesc'].startswith('milo_'))|df['SKUDesc'].startswith('maggi_')|df['SKUDesc'].startswith('ncf_')).select(['SKUDesc','ProvinceCode','SubChannelDesc','Final_Rank'])

In [5]:
df.filter(df['SKUDesc'].startswith('milo_')).count()

2110

In [6]:
df_large.show()

+--------------------+------------+------------------+----------+
|             SKUDesc|ProvinceCode|    SubChannelDesc|Final_Rank|
+--------------------+------------+------------------+----------+
|      milo_uht_180ml|           3|01.Large Provision|         1|
|milo_uhtlesssugar...|           3|01.Large Provision|         2|
|      milo_uht_115ml|           3|01.Large Provision|         3|
| maggi_l_oyster_350g|           3|01.Large Provision|         4|
| maggi_l_oyster_820g|           3|01.Large Provision|         5|
|   milo_reg_jar_400g|           3|01.Large Provision|         6|
|   ncf_3in1_strsib20|           3|01.Large Provision|         8|
|      milo_uht_180ml|           4|01.Large Provision|         1|
|      milo_uht_115ml|           4|01.Large Provision|         2|
|milo_uhtlesssugar...|           4|01.Large Provision|         3|
| maggi_l_oyster_820g|           4|01.Large Provision|         4|
| maggi_l_oyster_350g|           4|01.Large Provision|         5|
|   milo_r

In [7]:
df_large.count()

107

In [8]:
df_large = df.filter((df['SubChannelDesc']=='01.Large Provision')&(df['Final_Rank']<=8))

In [9]:
df_large.show()

+--------------------+---------+--------+------------+------------+----------------+------------------+----------+---------+-----------------+----------------------+-------------------+-----------------+----------+
|             SKUDesc|SubChanel|SKULabel|ProvinceCode|GroupChannel|model_prediction|    SubChannelDesc|Start_date|Pred_Rank|Profitablity_Rank|Normalize_Profitablity|Normalize_Pred_rank|   Composite_Rank|Final_Rank|
+--------------------+---------+--------+------------+------------+----------------+------------------+----------+---------+-----------------+----------------------+-------------------+-----------------+----------+
|      milo_uht_180ml|      101|   PC185|           3|          TT|89.6886934037138|01.Large Provision|2023-12-01|        1|                1|     -2.37668858890866|  -1.70676403707744|-1.77375649226056|         1|
|milo_uhtlesssugar...|      101|   PC191|           3|          TT|83.0023916738447|01.Large Provision|2023-12-01|        2|                

In [10]:
from pyspark.sql.functions import when
df = df.withColumn('Brand', when(df['SKUDesc'].startswith('maggi_'), 'maggi').when(df['SKUDesc'].startswith('milo_'), 'milo').when(df['SKUDesc'].startswith('ncf_'), 'ncf'))

In [11]:
df.select(['SKUDesc', 'Brand']).show()

+--------------------+-----+
|             SKUDesc|Brand|
+--------------------+-----+
|      milo_uht_180ml| milo|
|      milo_uht_115ml| milo|
|milo_uhtlesssugar...| milo|
|   milo_reg_jar_400g| milo|
|       milo_3in1_sic| milo|
|nangum_rtd_grow_1...| null|
|  nanoptipro_s2_400g| null|
|nestleyoguyogurt_...| null|
|milo_rtdsigactivg...| milo|
|nestleyoguyogurt_...| null|
|       nesvita_bib20| null|
|  nanoptipro_s2_900g| null|
|  nanoptipro_p3_850g| null|
|          al110_400g| null|
|   milo_rtdtin_240ml| milo|
|  nanoptipro_p1_800g| null|
|milo_uhtlesssugar...| milo|
|nestleyoguyogurt_...| null|
|nangum_rtd_hmo_115ml| null|
|naninfinipro_s1_800g| null|
+--------------------+-----+
only showing top 20 rows



In [12]:
df.filter(df['SubChannelDesc']=='01.Large Provision').count()

706

In [13]:
df.columns

['SKUDesc',
 'SubChanel',
 'SKULabel',
 'ProvinceCode',
 'GroupChannel',
 'model_prediction',
 'SubChannelDesc',
 'Start_date',
 'Pred_Rank',
 'Profitablity_Rank',
 'Normalize_Profitablity',
 'Normalize_Pred_rank',
 'Composite_Rank',
 'Final_Rank',
 'Brand']

In [14]:
df.filter((df['SubChannelDesc']=='01.Large Provision')&(df['Final_Rank']<=8)).select(['SKUDesc','ProvinceCode','SubChannelDesc','Final_Rank','Brand']).show()

+--------------------+------------+------------------+----------+-----+
|             SKUDesc|ProvinceCode|    SubChannelDesc|Final_Rank|Brand|
+--------------------+------------+------------------+----------+-----+
|      milo_uht_180ml|           3|01.Large Provision|         1| milo|
|milo_uhtlesssugar...|           3|01.Large Provision|         2| milo|
|      milo_uht_115ml|           3|01.Large Provision|         3| milo|
| maggi_l_oyster_350g|           3|01.Large Provision|         4|maggi|
| maggi_l_oyster_820g|           3|01.Large Provision|         5|maggi|
|   milo_reg_jar_400g|           3|01.Large Provision|         6| milo|
|    nestea_lemon_sib|           3|01.Large Provision|         7| null|
|   ncf_3in1_strsib20|           3|01.Large Provision|         8|  ncf|
|      milo_uht_180ml|           4|01.Large Provision|         1| milo|
|      milo_uht_115ml|           4|01.Large Provision|         2| milo|
|milo_uhtlesssugar...|           4|01.Large Provision|         3

In [15]:
df.filter(df['SubChannelDesc']=='01.Large Provision').groupby('Brand').count().show()

+-----+-----+
|Brand|count|
+-----+-----+
|  ncf|  133|
| null|  190|
|maggi|  226|
| milo|  157|
+-----+-----+



In [16]:
df.count()

10000

In [17]:
df = df.withColumn('Is_Top_Brand', when(df['SKUDesc'].startswith('maggi_'), 'yes').when(df['SKUDesc'].startswith('milo_'), 'yes').when(df['SKUDesc'].startswith('ncf_'), 'yes').otherwise('no'))

In [18]:
df_filter = df.filter((df['SubChannelDesc']=='01.Large Provision')&(df['Final_Rank']<=8)).select(['SKUDesc','ProvinceCode','SubChannelDesc','Final_Rank','Brand','Is_Top_Brand'])
df_filter.show()

+--------------------+------------+------------------+----------+-----+------------+
|             SKUDesc|ProvinceCode|    SubChannelDesc|Final_Rank|Brand|Is_Top_Brand|
+--------------------+------------+------------------+----------+-----+------------+
|      milo_uht_180ml|           3|01.Large Provision|         1| milo|         yes|
|milo_uhtlesssugar...|           3|01.Large Provision|         2| milo|         yes|
|      milo_uht_115ml|           3|01.Large Provision|         3| milo|         yes|
| maggi_l_oyster_350g|           3|01.Large Provision|         4|maggi|         yes|
| maggi_l_oyster_820g|           3|01.Large Provision|         5|maggi|         yes|
|   milo_reg_jar_400g|           3|01.Large Provision|         6| milo|         yes|
|    nestea_lemon_sib|           3|01.Large Provision|         7| null|          no|
|   ncf_3in1_strsib20|           3|01.Large Provision|         8|  ncf|         yes|
|      milo_uht_180ml|           4|01.Large Provision|         1|

In [19]:
province_codes = list(set([int(i['ProvinceCode']) for i in df_filter.collect()]))
province_codes = sorted(province_codes)
province_codes

[3, 4, 6, 10, 11, 12, 17, 18, 23, 27, 29, 30, 32, 33]

In [20]:
top_brands = ['maggi', 'milo', 'ncf']

In [75]:
columns = df.columns
for i in province_codes:
    df_val = df.filter((df['SubChannelDesc']=='01.Large Provision')&(df['ProvinceCode']==str(i))&(df['Final_Rank']<=7))
    # df_val.select(['SKUDesc','ProvinceCode','SubChannelDesc','Final_Rank','Brand','Is_Top_Brand']).show(8)
    all_brands = list(set([i['Brand'] for i in df_val.collect()[:7]]))
    print('all_brands: ', all_brands)
    if None in all_brands:
        all_brands.remove(None)
    if (len(all_brands) != 3):
        print(f'Province code {i} does not contain all brands {all_brands}')
        if 'ncf' not in all_brands:
            df_val1 = df.filter((df['SubChannelDesc']=='01.Large Provision')&(df['Brand']=='ncf')&(df['ProvinceCode']==str(i)))
            # df_val.select(['SKUDesc','ProvinceCode','SubChannelDesc','Final_Rank','Brand']).show(1)
            row_val = df_val1.collect()[0]
            # row_val['Final_Rank'] = 8
            # print(f'ncf_row: {row_val}')
            new_row = spark.createDataFrame([row_val], columns)
            df_val = df_val.union(new_row)
            # print(df_val.collect()[:8])
            df_val.select(['SKUDesc','ProvinceCode','SubChannelDesc','Final_Rank','Brand']).show()
            #display(df_val.toPandas())
            #df_val.write.csv('sample.csv', header=True)

all_brands:  ['maggi', 'milo', None]
Province code 3 does not contain all brands ['maggi', 'milo']
+--------------------+------------+------------------+----------+-----+
|             SKUDesc|ProvinceCode|    SubChannelDesc|Final_Rank|Brand|
+--------------------+------------+------------------+----------+-----+
|      milo_uht_180ml|           3|01.Large Provision|         1| milo|
|milo_uhtlesssugar...|           3|01.Large Provision|         2| milo|
|      milo_uht_115ml|           3|01.Large Provision|         3| milo|
| maggi_l_oyster_350g|           3|01.Large Provision|         4|maggi|
| maggi_l_oyster_820g|           3|01.Large Provision|         5|maggi|
|   milo_reg_jar_400g|           3|01.Large Provision|         6| milo|
|    nestea_lemon_sib|           3|01.Large Provision|         7| null|
|   ncf_3in1_strsib20|           3|01.Large Provision|         8|  ncf|
+--------------------+------------+------------------+----------+-----+

all_brands:  ['maggi', 'milo']
Provi

In [53]:
pandas_df = df.toPandas()

In [29]:
#len(df.filter(df['Brand']=='maggi').collect()[:8])

In [28]:
#df.toPandas()

In [54]:
pandas_df[(pandas_df['SubChannelDesc'] == '01.Large Provision') & (pandas_df['ProvinceCode'] == '4')&(pandas_df['Brand'] == 'ncf')]

,SKUDesc,SubChanel,SKULabel,ProvinceCode,GroupChannel,model_prediction,SubChannelDesc,Start_date,Pred_Rank,Profitablity_Rank,Normalize_Profitablity,Normalize_Pred_rank,Composite_Rank,Final_Rank,Brand,Is_Top_Brand
1935,ncf_cfv_sib15,101,PC249,4,TT,20.3703703703704,01.Large Provision,2023-12-01,13,1,-2.37668858890866,-0.592748978363819,-0.771142939418303,11,ncf,yes
1939,ncf_3in1_strsib20,101,PC244,4,TT,16.6666666666667,01.Large Provision,2023-12-01,18,1,-2.37668858890866,-0.136788225776266,-0.360778262089506,15,ncf,yes
1941,ncf_suada_sib,101,PC269,4,TT,12.962962962963,01.Large Provision,2023-12-01,19,1,-2.37668858890866,-0.045596075258755,-0.278705326623746,17,ncf,yes
1944,ncf_3in1_regsib20,101,PC240,4,TT,11.1111111111111,01.Large Provision,2023-12-01,23,1,-2.37668858890866,0.319172526811287,0.049586415239292,20,ncf,yes
1948,ncf_rtd_180ml_espresso,101,PC265,4,TT,11.1111111111111,01.Large Provision,2023-12-01,22,2,0.420753482246988,0.227980376293777,0.247257686889098,24,ncf,yes
1951,ncf_rtd_180ml_cfv,101,PC264,4,TT,7.40740740740741,01.Large Provision,2023-12-01,26,2,0.420753482246988,0.592748978363819,0.575549428752136,27,ncf,yes
1954,ncf_rtd_180ml_cfsd,101,PC263,4,TT,5.55555555555556,01.Large Provision,2023-12-01,30,2,0.420753482246988,0.957517580433862,0.903841170615174,30,ncf,yes


In [48]:
top_ranks = [str(i) for i in range(1,9)]
pandas_df[(pandas_df['SubChannelDesc'] == '01.Large Provision')& (pandas_df['Final_Rank'].isin(top_ranks)) & (pandas_df['ProvinceCode'] == '4')]

,SKUDesc,SubChanel,SKULabel,ProvinceCode,GroupChannel,model_prediction,SubChannelDesc,Start_date,Pred_Rank,Profitablity_Rank,Normalize_Profitablity,Normalize_Pred_rank,Composite_Rank,Final_Rank,Brand,Is_Top_Brand
1925,milo_uht_180ml,101,PC185,4,TT,84.6132893354516,01.Large Provision,2023-12-01,1,1,-2.37668858890866,-1.68705478457395,-1.75601816500742,1,milo,yes
1926,milo_uht_115ml,101,PC184,4,TT,82.0492359285843,01.Large Provision,2023-12-01,2,1,-2.37668858890866,-1.59586263405644,-1.67394522954166,2,milo,yes
1927,milo_uhtlesssugar_180ml,101,PC191,4,TT,61.8429566329404,01.Large Provision,2023-12-01,3,1,-2.37668858890866,-1.50467048353893,-1.5918722940759,3,milo,yes
1928,maggi_l_oyster_820g,101,PC146,4,TT,54.9596159484236,01.Large Provision,2023-12-01,4,1,-2.37668858890866,-1.41347833302141,-1.50979935861014,4,maggi,yes
1929,maggi_l_oyster_350g,101,PC144,4,TT,49.7343051435888,01.Large Provision,2023-12-01,5,1,-2.37668858890866,-1.3222861825039,-1.42772642314438,5,maggi,yes
1930,milo_reg_jar_400g,101,PC178,4,TT,48.852032160332,01.Large Provision,2023-12-01,6,1,-2.37668858890866,-1.23109403198639,-1.34565348767862,6,milo,yes
1931,milo_3in1_sic,101,PC175,4,TT,45.3056437123634,01.Large Provision,2023-12-01,7,1,-2.37668858890866,-1.13990188146888,-1.26358055221286,7,milo,yes
1932,nestea_lemon_sib,101,PC275,4,TT,35.6911208987759,01.Large Provision,2023-12-01,9,1,-2.37668858890866,-0.957517580433862,-1.09943468128134,8,None,no


In [55]:
pandas_df.iloc[1932]

SKUDesc                     nestea_lemon_sib
SubChanel                                101
SKULabel                               PC275
ProvinceCode                               4
GroupChannel                              TT
model_prediction            35.6911208987759
SubChannelDesc            01.Large Provision
Start_date                        2023-12-01
Pred_Rank                                  9
Profitablity_Rank                          1
Normalize_Profitablity     -2.37668858890866
Normalize_Pred_rank       -0.957517580433862
Composite_Rank             -1.09943468128134
Final_Rank                                 8
Brand                                   None
Is_Top_Brand                              no
Name: 1932, dtype: object

In [56]:
pandas_df.iloc[1935]

SKUDesc                        ncf_cfv_sib15
SubChanel                                101
SKULabel                               PC249
ProvinceCode                               4
GroupChannel                              TT
model_prediction            20.3703703703704
SubChannelDesc            01.Large Provision
Start_date                        2023-12-01
Pred_Rank                                 13
Profitablity_Rank                          1
Normalize_Profitablity     -2.37668858890866
Normalize_Pred_rank       -0.592748978363819
Composite_Rank            -0.771142939418303
Final_Rank                                11
Brand                                    ncf
Is_Top_Brand                             yes
Name: 1935, dtype: object

In [57]:
row_1932_copy = pandas_df.iloc[1932].copy()
row_1935_copy = pandas_df.iloc[1935].copy() 
pandas_df.iloc[1932] =  row_1935_copy
pandas_df.iloc[1935] =  row_1932_copy

In [59]:
pandas_df.iloc[1935]

SKUDesc                     nestea_lemon_sib
SubChanel                                101
SKULabel                               PC275
ProvinceCode                               4
GroupChannel                              TT
model_prediction            35.6911208987759
SubChannelDesc            01.Large Provision
Start_date                        2023-12-01
Pred_Rank                                  9
Profitablity_Rank                          1
Normalize_Profitablity     -2.37668858890866
Normalize_Pred_rank       -0.957517580433862
Composite_Rank             -1.09943468128134
Final_Rank                                 8
Brand                                   None
Is_Top_Brand                              no
Name: 1935, dtype: object

In [62]:
pandas_df.iloc[1932]['Final_Rank'] = 8

/tmp/ipykernel_5744/3420898625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_df.iloc[1932]['Final_Rank'] = 8


In [64]:
top_ranks = [str(i) for i in range(1,9)]
pandas_df[(pandas_df['SubChannelDesc'] == '01.Large Provision')& (pandas_df['Brand'].isin(top_brands)) & (pandas_df['Final_Rank'].isin(top_ranks)) & (pandas_df['ProvinceCode'] == '4')]

,SKUDesc,SubChanel,SKULabel,ProvinceCode,GroupChannel,model_prediction,SubChannelDesc,Start_date,Pred_Rank,Profitablity_Rank,Normalize_Profitablity,Normalize_Pred_rank,Composite_Rank,Final_Rank,Brand,Is_Top_Brand
1925,milo_uht_180ml,101,PC185,4,TT,84.6132893354516,01.Large Provision,2023-12-01,1,1,-2.37668858890866,-1.68705478457395,-1.75601816500742,1,milo,yes
1926,milo_uht_115ml,101,PC184,4,TT,82.0492359285843,01.Large Provision,2023-12-01,2,1,-2.37668858890866,-1.59586263405644,-1.67394522954166,2,milo,yes
1927,milo_uhtlesssugar_180ml,101,PC191,4,TT,61.8429566329404,01.Large Provision,2023-12-01,3,1,-2.37668858890866,-1.50467048353893,-1.5918722940759,3,milo,yes
1928,maggi_l_oyster_820g,101,PC146,4,TT,54.9596159484236,01.Large Provision,2023-12-01,4,1,-2.37668858890866,-1.41347833302141,-1.50979935861014,4,maggi,yes
1929,maggi_l_oyster_350g,101,PC144,4,TT,49.7343051435888,01.Large Provision,2023-12-01,5,1,-2.37668858890866,-1.3222861825039,-1.42772642314438,5,maggi,yes
1930,milo_reg_jar_400g,101,PC178,4,TT,48.852032160332,01.Large Provision,2023-12-01,6,1,-2.37668858890866,-1.23109403198639,-1.34565348767862,6,milo,yes
1931,milo_3in1_sic,101,PC175,4,TT,45.3056437123634,01.Large Provision,2023-12-01,7,1,-2.37668858890866,-1.13990188146888,-1.26358055221286,7,milo,yes


In [63]:
pandas_df.iloc[1932]

SKUDesc                        ncf_cfv_sib15
SubChanel                                101
SKULabel                               PC249
ProvinceCode                               4
GroupChannel                              TT
model_prediction            20.3703703703704
SubChannelDesc            01.Large Provision
Start_date                        2023-12-01
Pred_Rank                                 13
Profitablity_Rank                          1
Normalize_Profitablity     -2.37668858890866
Normalize_Pred_rank       -0.592748978363819
Composite_Rank            -0.771142939418303
Final_Rank                                11
Brand                                    ncf
Is_Top_Brand                             yes
Name: 1932, dtype: object

In [65]:
pandas_df.to_csv('output.csv', sep='\t', encoding='utf-8')

In [66]:
top_ranks = [str(i) for i in range(1,9)]
pandas_df[(pandas_df['SubChannelDesc'] == '01.Large Provision')& (pandas_df['Brand'].isin(top_brands)) & (pandas_df['Final_Rank'].isin(top_ranks)) & (pandas_df['ProvinceCode'] == '6')]

,SKUDesc,SubChanel,SKULabel,ProvinceCode,GroupChannel,model_prediction,SubChannelDesc,Start_date,Pred_Rank,Profitablity_Rank,Normalize_Profitablity,Normalize_Pred_rank,Composite_Rank,Final_Rank,Brand,Is_Top_Brand
2244,milo_uht_180ml,101,PC185,6,TT,89.7329877711747,01.Large Provision,2023-12-01,1,1,-2.37668858890866,-1.69397910771984,-1.76225005583872,1,milo,yes
2245,milo_uht_115ml,101,PC184,6,TT,82.669145942783,01.Large Provision,2023-12-01,2,1,-2.37668858890866,-1.61698005736894,-1.69295091052291,2,milo,yes
2246,milo_uhtlesssugar_180ml,101,PC191,6,TT,80.9993396902754,01.Large Provision,2023-12-01,3,1,-2.37668858890866,-1.53998100701804,-1.6236517652071,3,milo,yes
2248,maggi_l_oyster_820g,101,PC146,6,TT,60.2629888658786,01.Large Provision,2023-12-01,5,1,-2.37668858890866,-1.38598290631623,-1.48505347457548,5,maggi,yes
2249,maggi_l_oyster_350g,101,PC144,6,TT,58.2482174177432,01.Large Provision,2023-12-01,6,1,-2.37668858890866,-1.30898385596533,-1.41575432925966,6,maggi,yes
2250,milo_rtdtin_240ml,101,PC183,6,TT,51.5788722832173,01.Large Provision,2023-12-01,7,1,-2.37668858890866,-1.23198480561443,-1.34645518394385,7,milo,yes
2251,milo_reg_jar_400g,101,PC178,6,TT,49.0976567863751,01.Large Provision,2023-12-01,8,1,-2.37668858890866,-1.15498575526353,-1.27715603862804,8,milo,yes


In [67]:
pandas_df.iloc[2247]

SKUDesc                     nestea_lemon_sib
SubChanel                                101
SKULabel                               PC275
ProvinceCode                               6
GroupChannel                              TT
model_prediction            62.4033833749811
SubChannelDesc            01.Large Provision
Start_date                        2023-12-01
Pred_Rank                                  4
Profitablity_Rank                          1
Normalize_Profitablity     -2.37668858890866
Normalize_Pred_rank        -1.46298195666713
Composite_Rank             -1.55435261989129
Final_Rank                                 4
Brand                                   None
Is_Top_Brand                              no
Name: 2247, dtype: object

In [71]:
pandas_df[(pandas_df['SubChannelDesc'] == '01.Large Provision') & (pandas_df['ProvinceCode'] == '6')&(pandas_df['Brand'] == 'ncf')]

,SKUDesc,SubChanel,SKULabel,ProvinceCode,GroupChannel,model_prediction,SubChannelDesc,Start_date,Pred_Rank,Profitablity_Rank,Normalize_Profitablity,Normalize_Pred_rank,Composite_Rank,Final_Rank,Brand,Is_Top_Brand
2247,ncf_3in1_strsib20,101,PC244,6,TT,41.4792469882798,01.Large Provision,2023-12-01,9,1,-2.37668858890866,-1.07798670491263,-1.20785689331223,9,ncf,yes
2253,ncf_cfv_sib15,101,PC249,6,TT,41.0083338826764,01.Large Provision,2023-12-01,11,1,-2.37668858890866,-0.923988604210822,-1.06925860268061,10,ncf,yes
2254,ncf_3in1_regsib20,101,PC240,6,TT,34.3516204341615,01.Large Provision,2023-12-01,13,1,-2.37668858890866,-0.769990503509018,-0.930660312048982,11,ncf,yes
2260,ncf_suada_sib,101,PC269,6,TT,15.2847158906904,01.Large Provision,2023-12-01,20,1,-2.37668858890866,-0.230997151052705,-0.445566294838301,17,ncf,yes
2265,ncf_rtd_180ml_espresso,101,PC265,6,TT,15.8227848101266,01.Large Provision,2023-12-01,19,2,0.420753482246988,-0.307996201403607,-0.235121233038548,22,ncf,yes
2271,ncf_rtd_180ml_cfsd,101,PC263,6,TT,7.6517364492048,01.Large Provision,2023-12-01,27,2,0.420753482246988,0.307996201403607,0.319271929487945,28,ncf,yes
2273,ncf_rtd_180ml_cfv,101,PC264,6,TT,5.09574813372282,01.Large Provision,2023-12-01,30,2,0.420753482246988,0.538993352456313,0.52716936543538,30,ncf,yes
2279,ncf_rtd_180ml_latte,101,PC266,6,TT,2.55598831548199,01.Large Provision,2023-12-01,36,2,0.420753482246988,1.00098765456172,0.94296423733025,36,ncf,yes


In [69]:
pandas_df.iloc[2252]

SKUDesc                    ncf_3in1_strsib20
SubChanel                                101
SKULabel                               PC244
ProvinceCode                               6
GroupChannel                              TT
model_prediction            41.4792469882798
SubChannelDesc            01.Large Provision
Start_date                        2023-12-01
Pred_Rank                                  9
Profitablity_Rank                          1
Normalize_Profitablity     -2.37668858890866
Normalize_Pred_rank        -1.07798670491263
Composite_Rank             -1.20785689331223
Final_Rank                                 9
Brand                                    ncf
Is_Top_Brand                             yes
Name: 2252, dtype: object

In [70]:
row_2247_copy = pandas_df.iloc[2247].copy()
row_2252_copy = pandas_df.iloc[2252].copy() 
pandas_df.iloc[2247] =  row_2252_copy
pandas_df.iloc[2252] =  row_2247_copy

In [72]:
pandas_df.to_csv('output.csv', sep='\t', encoding='utf-8')

In [73]:
pandas_df.count()

SKUDesc                   10000
SubChanel                 10000
SKULabel                  10000
ProvinceCode              10000
GroupChannel              10000
model_prediction          10000
SubChannelDesc            10000
Start_date                10000
Pred_Rank                 10000
Profitablity_Rank         10000
Normalize_Profitablity    10000
Normalize_Pred_rank       10000
Composite_Rank            10000
Final_Rank                10000
Brand                      7017
Is_Top_Brand              10000
dtype: int64

In [76]:
top_ranks = [str(i) for i in range(1,8)]
pandas_df[(pandas_df['SubChannelDesc'] == '01.Large Provision')& (pandas_df['Final_Rank'].isin(top_ranks)) & (pandas_df['ProvinceCode'] == '4')]

,SKUDesc,SubChanel,SKULabel,ProvinceCode,GroupChannel,model_prediction,SubChannelDesc,Start_date,Pred_Rank,Profitablity_Rank,Normalize_Profitablity,Normalize_Pred_rank,Composite_Rank,Final_Rank,Brand,Is_Top_Brand
1925,milo_uht_180ml,101,PC185,4,TT,84.6132893354516,01.Large Provision,2023-12-01,1,1,-2.37668858890866,-1.68705478457395,-1.75601816500742,1,milo,yes
1926,milo_uht_115ml,101,PC184,4,TT,82.0492359285843,01.Large Provision,2023-12-01,2,1,-2.37668858890866,-1.59586263405644,-1.67394522954166,2,milo,yes
1927,milo_uhtlesssugar_180ml,101,PC191,4,TT,61.8429566329404,01.Large Provision,2023-12-01,3,1,-2.37668858890866,-1.50467048353893,-1.5918722940759,3,milo,yes
1928,maggi_l_oyster_820g,101,PC146,4,TT,54.9596159484236,01.Large Provision,2023-12-01,4,1,-2.37668858890866,-1.41347833302141,-1.50979935861014,4,maggi,yes
1929,maggi_l_oyster_350g,101,PC144,4,TT,49.7343051435888,01.Large Provision,2023-12-01,5,1,-2.37668858890866,-1.3222861825039,-1.42772642314438,5,maggi,yes
1930,milo_reg_jar_400g,101,PC178,4,TT,48.852032160332,01.Large Provision,2023-12-01,6,1,-2.37668858890866,-1.23109403198639,-1.34565348767862,6,milo,yes
1931,milo_3in1_sic,101,PC175,4,TT,45.3056437123634,01.Large Provision,2023-12-01,7,1,-2.37668858890866,-1.13990188146888,-1.26358055221286,7,milo,yes


In [83]:
pandas_df[(pandas_df['SubChannelDesc']=='01.Large Provision')&(pandas_df['Brand']=='maggi')&(pandas_df['ProvinceCode']=='17')]

,SKUDesc,SubChanel,SKULabel,ProvinceCode,GroupChannel,model_prediction,SubChannelDesc,Start_date,Pred_Rank,Profitablity_Rank,Normalize_Profitablity,Normalize_Pred_rank,Composite_Rank,Final_Rank,Brand,Is_Top_Brand
4755,maggi_l_oyster_820g,101,PC146,17,TT,44.8468452792486,01.Large Provision,2023-12-01,8,1,-2.37668858890866,-1.25988157669742,-1.37156227791855,8,maggi,yes
4757,maggi_l_soyacondensed_700ml,101,PC155,17,TT,40.1660108452994,01.Large Provision,2023-12-01,10,1,-2.37668858890866,-1.13389341902768,-1.25817293601578,10,maggi,yes
4758,maggi_l_oyster_350g,101,PC144,17,TT,33.8256008683345,01.Large Provision,2023-12-01,11,1,-2.37668858890866,-1.07089934019281,-1.2014782650644,11,maggi,yes
4761,maggi_l_soyacondensed_300ml,101,PC154,17,TT,12.8946849908872,01.Large Provision,2023-12-01,17,1,-2.37668858890866,-0.692934867183583,-0.861310239356091,14,maggi,yes
4765,maggi_b_mushroom_450g,101,PC129,17,TT,11.0575579323533,01.Large Provision,2023-12-01,20,1,-2.37668858890866,-0.50395263067897,-0.691226226501939,18,maggi,yes
4768,maggi_l_soya_300ml,101,PC148,17,TT,10.1743756465199,01.Large Provision,2023-12-01,22,1,-2.37668858890866,-0.377964473009227,-0.577836884599171,21,maggi,yes
4771,maggi_l_soya_700ml,101,PC150,17,TT,9.9305452933353,01.Large Provision,2023-12-01,24,1,-2.37668858890866,-0.251976315339485,-0.464447542696403,24,maggi,yes
4773,maggi_b_mushroom_200g,101,PC127,17,TT,7.50768769219231,01.Large Provision,2023-12-01,29,1,-2.37668858890866,0.062994078834871,-0.180974187939482,26,maggi,yes
4774,maggi_b_pork_2kg,101,PC132,17,TT,9.42441259051278,01.Large Provision,2023-12-01,25,2,0.420753482246988,-0.188982236504614,-0.128008664629453,27,maggi,yes
4783,maggi_l_oyster_150g,101,PC142,17,TT,1.99512413552913,01.Large Provision,2023-12-01,39,1,-2.37668858890866,0.692934867183583,0.385972521574359,36,maggi,yes
